In [1]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@2c71dfad

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [3]:
val spark = SparkSession.builder.appName("DI Lab5 ").getOrCreate()

spark = org.apache.spark.sql.SparkSession@2c71dfad


org.apache.spark.sql.SparkSession@2c71dfad

# Data preparation

In [5]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/denis.ivashchenko/lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [6]:
val categoriesCols = List(
    "REST_AVG_CUR",
    "REST_AVG_PAYM",
    "TRANS_COUNT_ATM_PRC",
    "AMOUNT_RUB_ATM_PRC",
    "CR_PROD_CNT_TOVR",
    "REST_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_CUR_3M", 
    "CNT_TRAN_ATM_TENDENCY3M", 
    "CNT_TRAN_ATM_TENDENCY1M",
    "SUM_TRAN_ATM_TENDENCY3M",
    "SUM_TRAN_ATM_TENDENCY1M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "TRANS_AMOUNT_TENDENCY3M",
    "TURNOVER_DYNAMIC_CC_3M",
    "TARGET"
)

categoriesCols = List(REST_AVG_CUR, REST_AVG_PAYM, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, CR_PROD_CNT_TOVR, REST_DYNAMIC_CUR_1M, REST_DYNAMIC_CUR_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_CUR_3M, TRANS_AMOUNT_TENDENCY3M, TURNOVER_DYNAMIC_CC_3M, TARGET)


List(REST_AVG_CUR, REST_AVG_PAYM, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, CR_PROD_CNT_TOVR, REST_DYNAMIC_CUR_1M, REST_DYNAMIC_CUR_3M, CNT_TRAN_ATM_TENDENCY3M, CNT_TRAN_ATM_TENDENCY1M, SUM_TRAN_ATM_TENDENCY3M, SUM_TRAN_ATM_TENDENCY1M, TURNOVER_DYNAMIC_CUR_3M, TRANS_AMOUNT_TENDENCY3M, TURNOVER_DYNAMIC_CC_3M, TARGET)

In [7]:
trainDf = trainDf.select($"REST_AVG_CUR",
                         $"REST_AVG_PAYM",
                        $"TRANS_COUNT_ATM_PRC",
    $"CR_PROD_CNT_TOVR",
    $"AMOUNT_RUB_ATM_PRC",
    $"REST_DYNAMIC_CUR_1M",
    $"REST_DYNAMIC_CUR_3M", 
    $"CNT_TRAN_ATM_TENDENCY3M", 
    $"CNT_TRAN_ATM_TENDENCY1M",
    $"SUM_TRAN_ATM_TENDENCY3M",
    $"SUM_TRAN_ATM_TENDENCY1M",
    $"TURNOVER_DYNAMIC_CUR_3M",
    $"TRANS_AMOUNT_TENDENCY3M",
    $"TURNOVER_DYNAMIC_CC_3M",
    $"TARGET").na.fill(0)

trainDf = [REST_AVG_CUR: double, REST_AVG_PAYM: double ... 13 more fields]


[REST_AVG_CUR: double, REST_AVG_PAYM: double ... 13 more fields]

In [8]:
trainDf.show(1, 0, true)

-RECORD 0--------------------------------------
 REST_AVG_CUR            | 19069.6917166667    
 REST_AVG_PAYM           | 0.0                 
 TRANS_COUNT_ATM_PRC     | 0.981818181818182   
 CR_PROD_CNT_TOVR        | 1                   
 AMOUNT_RUB_ATM_PRC      | 0.9264718189135429  
 REST_DYNAMIC_CUR_1M     | 0.139327197286461   
 REST_DYNAMIC_CUR_3M     | 0.24404241658152398 
 CNT_TRAN_ATM_TENDENCY3M | 0.351851851851852   
 CNT_TRAN_ATM_TENDENCY1M | 0.166666666666667   
 SUM_TRAN_ATM_TENDENCY3M | 0.329320637235633   
 SUM_TRAN_ATM_TENDENCY1M | 0.16790105289165602 
 TURNOVER_DYNAMIC_CUR_3M | 0.30374965665416803 
 TRANS_AMOUNT_TENDENCY3M | 0.305106289785464   
 TURNOVER_DYNAMIC_CC_3M  | 0.0                 
 TARGET                  | 0                   
only showing top 1 row



In [9]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [REST_AVG_CUR: double, REST_AVG_PAYM: double ... 13 more fields]
trainDfTest = [REST_AVG_CUR: double, REST_AVG_PAYM: double ... 13 more fields]


[REST_AVG_CUR: double, REST_AVG_PAYM: double ... 13 more fields]

# Train model

In [10]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, LogisticRegression}
import org.apache.spark.ml.feature.{VectorAssembler}

In [11]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_d106dba2956f


vecAssembler_d106dba2956f

In [12]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [REST_AVG_CUR: double, REST_AVG_PAYM: double ... 14 more fields]


[REST_AVG_CUR: double, REST_AVG_PAYM: double ... 14 more fields]

In [18]:
val lr_model = new LogisticRegression()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    //.setNumTrees(10)
    .fit(trainDfCleaned)

lr_model = LogisticRegressionModel: uid = logreg_d54a122ede51, numClasses = 2, numFeatures = 14


LogisticRegressionModel: uid = logreg_d54a122ede51, numClasses = 2, numFeatures = 14

In [13]:
// Train RandomForest
val randomForestModel = new RandomForestClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(42)
    //.setNumTrees(50)
    .setMaxDepth(7)
    .setImpurity("gini")
    .fit(trainDfCleaned)

randomForestModel = RandomForestClassificationModel (uid=rfc_2e58fd1e4b93) with 20 trees


RandomForestClassificationModel (uid=rfc_2e58fd1e4b93) with 20 trees

In [14]:
val predictions = randomForestModel.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+----------+--------------------+--------------------+----------+
|TARGET|  features|       rawPrediction|         probability|prediction|
+------+----------+--------------------+--------------------+----------+
|     0|(14,[],[])|[17.5678185605611...|[0.87839092802805...|       0.0|
|     0|(14,[],[])|[17.5678185605611...|[0.87839092802805...|       0.0|
|     0|(14,[],[])|[17.5678185605611...|[0.87839092802805...|       0.0|
|     0|(14,[],[])|[17.5678185605611...|[0.87839092802805...|       0.0|
|     0|(14,[],[])|[17.5678185605611...|[0.87839092802805...|       0.0|
+------+----------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [REST_AVG_CUR: double, REST_AVG_PAYM: double ... 17 more fields]


[REST_AVG_CUR: double, REST_AVG_PAYM: double ... 17 more fields]

In [15]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0-----------------------------------------------
 probability | [0.8783909280280584,0.12160907197194151] 
only showing top 1 row



In [16]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [17]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [18]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.12160907197194151|
|   0.12160907197194151|
|   0.12160907197194151|
|   0.12160907197194151|
|   0.12160907197194151|
+----------------------+
only showing top 5 rows



In [19]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [20]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_2dc1cd1adaed
accuracy = 0.7813741051919764


0.7813741051919764

# Predict

In [24]:
var testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/denis.ivashchenko/lab05_test.csv")
    .na.fill(0)

testDfData = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [25]:
testDfData = testDfData.select($"ID",
    $"REST_AVG_CUR",
    $"REST_AVG_PAYM",
    $"TRANS_COUNT_ATM_PRC",
    $"CR_PROD_CNT_TOVR",
    $"AMOUNT_RUB_ATM_PRC",
    $"REST_DYNAMIC_CUR_1M",
    $"REST_DYNAMIC_CUR_3M", 
    $"CNT_TRAN_ATM_TENDENCY3M", 
    $"CNT_TRAN_ATM_TENDENCY1M",
    $"SUM_TRAN_ATM_TENDENCY3M",
    $"SUM_TRAN_ATM_TENDENCY1M",
    $"TURNOVER_DYNAMIC_CUR_3M",
    $"TRANS_AMOUNT_TENDENCY3M",
    $"TURNOVER_DYNAMIC_CC_3M"
                              )

testDfData = [ID: int, REST_AVG_CUR: double ... 13 more fields]


[ID: int, REST_AVG_CUR: double ... 13 more fields]

In [26]:
testDfData.count

44399

In [27]:
val vectorAssembler_test = new VectorAssembler()
    .setInputCols(testDfData.drop("ID").columns)
    .setOutputCol("features")

vectorAssembler_test = vecAssembler_1f0882001153


vecAssembler_1f0882001153

In [32]:
val realPredictions = randomForestModel.transform(vectorAssembler_test.transform(testDfData)).repartition(1).cache

realPredictions = [ID: int, REST_AVG_CUR: double ... 17 more fields]


[ID: int, REST_AVG_CUR: double ... 17 more fields]

In [29]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+--------------------+
|    ID|          prediction|
+------+--------------------+
|519130| 0.10629116206752565|
|234045|0.031064735664339305|
|401256| 0.03152485498962631|
|551070| 0.12163428815857662|
|367285|0.030208320470913864|
+------+--------------------+
only showing top 5 rows



In [33]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "target")
    .withColumnRenamed("ID", "id")
    .write
    .mode("overwrite")
    .format("csv")
    .option("header","True")
    .option("sep","\t")
    .save("/user/denis.ivashchenko/lab05.csv")

In [66]:
spark.stop()